### Submitted By: LALAM BHANU MANI KANTA NAIDU

# Task 1: Web Scraping to Gain Company Insights

---

## Web Scraping and Analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping Data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

## 1. Importing Required Libraries:

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

## 2. Scraping the reviews pages from the given website

In [2]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10 
page_size = 100

reviews = []
aircraft = []
seat_type = []
route = []
recommended = []
df = pd.DataFrame()

for i in range(1, pages + 1):
    
    rating = []
    category = []

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
        
    for para2 in parsed_content.find_all("div", {"class": "review-stats"}):
        for para3 in para2.find_all('td',{'class': 'review-value'}):
            rating.append(para3.get_text())
        recomend = rating[-1]
        rating = rating[:-1]
        
        for para4 in para2.find_all('td',{'class': 'review-rating-stars stars'}):
            para5 = len(para4.find_all('span', {'class' : 'star fill'}))
            rating.append(para5)
        rating.append(recomend)
    
        
        for para6 in para2.find_all('td',{'class': 'review-rating-header'}):
            category.append(para6.get_text())

        # Build the dataframe from the dictionary.
        data_dict = pd.DataFrame([rating], columns=category)
        df = pd.concat([df,data_dict], ignore_index=True).reset_index(drop=True)
        rating = []
        category = []
        
    print(f"--> {len(reviews)} total reviews")

df["reviews"] = reviews

Scraping page 1
--> 100 total reviews
Scraping page 2
--> 200 total reviews
Scraping page 3
--> 300 total reviews
Scraping page 4
--> 400 total reviews
Scraping page 5
--> 500 total reviews
Scraping page 6
--> 600 total reviews
Scraping page 7
--> 700 total reviews
Scraping page 8
--> 800 total reviews
Scraping page 9
--> 900 total reviews
Scraping page 10
--> 1000 total reviews


In [3]:
# Checking first 5 rows in dataframe:
df.head()

,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Wifi & Connectivity,Value For Money,Recommended,reviews
0,A350,Couple Leisure,Economy Class,Phoenix to Lisbon via London,February 2024,1.0,5.0,3.0,3.0,1.0,1.0,1,no,Not Verified | We have flown BA five times fr...
1,A320,Solo Leisure,Economy Class,London to Istanbul,February 2024,3.0,1.0,1.0,NaN,2.0,NaN,2,no,✅ Trip Verified | London Heathrow to Istanbul...
2,Boeing 777,Solo Leisure,Premium Economy,"Austin, TX to Shannon, Ireland via Austin",January 2024,3.0,5.0,3.0,3.0,4.0,NaN,2,no,"Not Verified | Jan 30th, I booked a last-minut..."
3,A380,Solo Leisure,First Class,London to San Francisco,November 2023,1.0,4.0,1.0,NaN,4.0,2.0,1,no,✅ Trip Verified | I am a British Airways Gold ...
4,NaN,Couple Leisure,Economy Class,London Heathrow to Casablanca,February 2024,NaN,NaN,NaN,NaN,1.0,NaN,1,no,Not Verified | Another case of reviewing Brit...


In [4]:
#checking data frame shape(No.of rows , No.of columns)
df.shape

(1000, 14)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Aircraft                533 non-null    object 
 1   Type Of Traveller       998 non-null    object 
 2   Seat Type               1000 non-null   object 
 3   Route                   997 non-null    object 
 4   Date Flown              1000 non-null   object 
 5   Seat Comfort            914 non-null    float64
 6   Cabin Staff Service     907 non-null    float64
 7   Food & Beverages        789 non-null    float64
 8   Inflight Entertainment  525 non-null    float64
 9   Ground Service          944 non-null    float64
 10  Wifi & Connectivity     295 non-null    float64
 11  Value For Money         1000 non-null   int64  
 12  Recommended             1000 non-null   object 
 13  reviews                 1000 non-null   object 
dtypes: float64(6), int64(1), object(7)
memory

In [6]:
# Saving collected data in csv format:
df.to_csv("BA_reviews.csv", index=False)

Now we have our dataset for this task! The loops above collected 1500 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!